# Blight violations Analysis

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Read the data form .csv file
DataFrame = pd.DataFrame(pd.read_csv('Blight_Violations.csv', low_memory = False))
#DataFrame.columns
#ataFrame.dtypes

### Data cleanup

In [4]:
# Generate a compliance variable that would be the predictive variable

def featureCleanup(DF):
    #drop the columns
    DF.drop(['Ticket Number'], axis = 1, inplace = True)
    DF.drop(['Violator Name'], axis = 1, inplace = True)
    DF.drop(['Violation Location'], axis =1, inplace = True)
    
    #convert dtypes appropirately    
    DF['Balance Due'] = DF['Balance Due'].str[1:].astype(np.float64)
    DF['Judgment Amount (Total Due)'] = DF['Judgment Amount (Total Due)'].str[1:].astype(np.float64)    
    
    #Remove all NaN in each column with 0
    for col in DF.columns:
        if(DF[col].dtypes == 'object'):
            DF.loc[DF[col].isnull(),col] = '0'
        if(DF[col].dtypes == 'int64'):
            DF.loc[DF[col].isnull(),col] = 0            
        if(DF[col].dtypes == 'float64'):
            print(col)
            DF.loc[DF[col].isnull(),col] = 0.0            
 
    return DF

DF = featureCleanup(DataFrame)





    

Judgment Amount (Total Due)
Balance Due
Violation Latitude
Violation Longitude


In [38]:
DF.loc[DF['Balance Due']!=DF['Judgment Amount (Total Due)'], ['Balance Due', 'Judgement Amount (Total Due)']].head()
#DF['Judgment Amount (Total Due)'].dtypes

,Balance Due,Judgement Amount (Total Due)
6,0,NaN
7,0,NaN
8,0,NaN
31,0,NaN
83,0,NaN


In [22]:
def populateCompliance(DF):
    #if payment status is 'Paid in full' then compliance is 0
    DF.loc[DF['Payment Status']=='PAID IN FULL','Compliance'] = 0
    DF.loc[DF['Payment Status']=='NO PAYMENT DUE' ,'Compliance'] = 0    
    
print(np.unique(DF['Payment Status']))

['0' 'NO PAYMENT APPLIED' 'NO PAYMENT DUE' 'PAID IN FULL'
 'PARTIAL PAYMENT APPLIED']
